<a href="https://colab.research.google.com/github/mugalan/google-api-python-wrapper/blob/main/API_Cookbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Dependencies

In [ ]:
import getpass, os, pathlib, textwrap, stat

TOKEN = getpass.getpass("GitHub token (hidden): ").strip()  # strip just in case

netrc = pathlib.Path("~/.netrc").expanduser()
netrc.write_text(textwrap.dedent(f"""
machine github.com
  login x-access-token
  password {TOKEN}
machine api.github.com
  login x-access-token
  password {TOKEN}
""").strip()+"\n", encoding="utf-8")

os.chmod(netrc, stat.S_IRUSR | stat.S_IWUSR)  # 600
print("~/.netrc configured.")
!git config --global --unset-all http.https://github.com/.extraheader || true
# installs your dev branch + ALL extras you defined
!python -m pip install --upgrade 'orise-one-client[all] @ git+https://github.com/mugalan/one-client.git@dev'

In [ ]:
# Robust upload to /content/utilities, then return to /content
from google.colab import files
import os, json

TARGET = "/content/utilities"

def safe_getcwd(default="/content"):
    try:
        return os.getcwd()
    except FileNotFoundError:
        # CWD vanished; reset to a known-good location
        os.chdir(default)
        return default

# Ensure we start from a valid place
orig_cwd = safe_getcwd("/content")

# Make sure target exists
os.makedirs(TARGET, exist_ok=True)

try:
    # Upload into the target by switching cwd
    os.chdir(TARGET)
    print(f"Uploading to: {os.getcwd()}")
    uploaded = files.upload()  # files land in /content/utilities

    # Optional: pretty-format JSONs in place
    for fname in uploaded.keys():
        try:
            with open(fname, "r+", encoding="utf-8") as f:
                obj = json.load(f)
                f.seek(0); json.dump(obj, f, ensure_ascii=False, indent=2); f.truncate()
            print(f"✓ formatted: {fname}")
        except Exception:
            # Not JSON? leave as-is.
            pass
finally:
    # Always try to go back; if orig_cwd is gone, fall back to /content
    try:
        os.chdir(orig_cwd)
    except Exception:
        os.chdir("/content")
    print("Back to:", safe_getcwd("/content"))

# Optional sanity check
# !ls -lh /content/utilities
# !ls -lh /content

#Installation

In [ ]:
from oriseoneclient import OriseOneClient, build_action_chain, plan_and_execute_task, run_new_chain_with_client, NodeSpec, Assistant, NodeManager, GraphRunInspector
from oriseoneclient.prelude import *
import getpass
openai_api_key = getpass.getpass("OpenAI key: ").strip()

#Tools

#Manual-Task-Execution

##Call Manual Task Execution

###Call DAG

###Initialize

In [ ]:
tools = [] #[UoP] #[TST]
llm = {'name': 'openAI_llm', 'data': {"api_key": openai_api_key, "model": "gpt-4.1"}}
oOne = OriseOneClient(tools=tools,llm=llm)
llm={"name":"openAI_llm","data":{"api_key":openai_api_key,"model":"gpt-4o-2024-08-06", "tools":[{"type": "web_search_preview"}]}}
node_manager=NodeManager(llm=llm)

In [ ]:
pd.DataFrame(oOne.tool_info)

In [ ]:
node_manager.add(adaptive_quiz_nodes)

In [ ]:
node_manager.graph_dependencies()

In [ ]:
node_manager.names()

In [ ]:
ME325-2025-Marks-All
get all sheets of the spreadsheet with name ME327-2025-Grades-All
for marks use grades-final,	for mapping use ILO-grade-map, use data_id data_33b33ed95d3e43b28f3af850a0a6f799
 and columns: ca-grades, end-grades, grades-final, studen-ILO-attainment, ILO-grade-map

###Execute chain

In [ ]:
items=[{'label':'L1', 'a':500, 'b':0.25},{'label':'L2', 'a':500, 'b':0.5},{'label':'L3', 'a':500, 'b':0.75}]
reset_response=oOne.tools[5].reset_estimator(items=items)

In [ ]:
plan=['recommend','adaptive_quizz_assitant','add_message','adaptive_quizz_assitant'] #['recommend'] #['reset_estimator'] #,
#['add_message','add_questions_to_google_form'] #['add_message','create_google_form'] #['add_message','get_gdrive_folder_explorer','add_message','get_all_sheets_in_a_google_sheet','add_message','get_ilo_attainment'] #['add_message','get_ilo_attainment'] #['add_message','get_gdrive_folder_explorer','add_message','get_all_sheets_in_a_google_sheet'] #['add_message','get_gdrive_folder_explorer','add_message','get_all_sheets_in_a_google_sheet'] #['add_message','get_ilo_attainment'] #,'add_message','general_chart_creator'] #['add_message','get_gdrive_folder_explorer','add_message','get_all_sheets_in_a_google_sheet'] #['add_message','read_data_from_file']#['add_message','general_chart_creator']#['add_message','get_gdrive_folder_explorer'] #['add_message','get_gdrive_folder_explorer','get_all_sheets_in_a_google_sheet','add_message','get_ilo_attainment','add_message','general_chart_creator'] #['add_message','general_chart_creator'] #,'get_ilo_attainment','general_chart_creator']#['add_message','get_ilo_attainment'] #['add_message','get_all_sheets_in_a_google_sheet'] #,'get_ilo_attainment','add_message','general_chart_creator'] #['add_message','general_chart_creator'] #["add_message","create_google_calendar_event"] #["add_message","send_email"]#["download_file_from_gdrive"] #
client=oOne

In [ ]:
oOne.thread

In [ ]:
client.thread=oOne.create_thread()
nodes=node_manager.list()
app = build_action_chain(nodes)

initial_state = {
    "plan": plan, # ordered chain
    "cursor": 0,
    "messages": [],        # optional; we don't write to it
    "node_outputs": {},    # will be filled with full per-node snapshots
}

result = await app.ainvoke(
    initial_state,
    config={"configurable": {
        "client": client,      # your persistent client to start from
        "thread_id": "thr-123",   # if you use a checkpointer
        "retry": {
            "max_attempts": 3,
            "backoff_s": 0.5,
            "multiplier": 2.0,
            "jitter": True,
            "on_error": "fail",
        }
        }},
)
viewer = GraphRunInspector(result=result, client=client)
#Message History

In [ ]:
running_thread=viewer.get_running_thread()
for msg in running_thread.get('context_texts',[]):
    print(msg)
#View tool outputs
run_nodes=viewer.node_names()
for node_name in run_nodes:
    tool_outputs=viewer.get_node_tool_results(node_name)
#Node execution timeline
execution_graph=viewer.plot_execution_graph()

###Output processing

In [ ]:
for data_all in viewer.get_all_data():
    print('data_id',data_all.get('id'))
    data=data_all.get('data')
    records=data.get('records')
    display(pd.DataFrame(records))

In [ ]:
viewer.get_total_llm_usage()

In [ ]:
# new_result=await run_new_chain_with_client_from_node(previous_result=result,available_nodes=nodes, last_node="get_gdrive_folder_explorer",new_nodes=["add_message", "download_file_from_gdrive"])

In [ ]:
[*result]

In [ ]:
viewer.node_names()

In [ ]:
node_name=viewer.node_names()[3]

In [ ]:
tool_outputs=viewer.get_node_tool_results(node_name)
records=tool_outputs.get('records')
figure=tool_outputs.get('figure')

In [ ]:
viewer.get_node_meta_data(node_name)

In [ ]:
tool_outputs

In [ ]:
if isinstance(tool_outputs,dict):
    records=tool_outputs.get('records')
    figure=tool_outputs.get('figure')

In [ ]:
node_meta_data=viewer.get_node_meta_data(node_name)

In [ ]:
[*node_meta_data]

In [ ]:
node_meta_data.get('assistant_meta_data')

In [ ]:
json.loads("{'label':'Geogrophy','outcome':0}")

In [ ]:
viewer.get_node_meta_data(node_name).get('response_text')

In [ ]:
viewer.get_node_assistant_data(1)       # by index

In [ ]:
full = viewer.node_snapshot(1)

In [ ]:
viewer.get_total_llm_usage()

In [ ]:
viewer.get_all_threads()

In [ ]:
running_thread=viewer.get_running_thread()
for msg in running_thread.get('context_texts',[]):
    print(msg)

In [ ]:
viewer.get_running_thread()

In [ ]:
viewer.get_node_client('download_file_from_gdrive').get_all_threads()

In [ ]:
snap_shots=oOne.snapshot_registry

In [ ]:
snap_shots

In [ ]:
# @title
snap_shots[2].get('client').client_id

In [ ]:
ILO_mapped_data=viewer.get_data_by_id('data_365aa4ff52294dab82a17c3657e8d38f')
ILO_mapped_data_df=pd.DataFrame(ILO_mapped_data.get('records'))

In [ ]:
ILO_mapped_data_df

In [ ]:
oOne.get_tool_method('principal_component_values')

In [ ]:
result2=oOne.tools[2].principal_component_values(cols=['ILO-1','ILO-2','ILO-3','ILO-4'], data=ILO_mapped_data)

In [ ]:
pca_data=result2.get('response',{}).get('data','{}')

In [ ]:
pd.DataFrame(json.loads(pca_data).get('records'))

In [ ]:
result3=oOne.tools[2].gmm_cluster_data(cols=['ILO-1','ILO-2','ILO-3','ILO-4'],n_components=2, data=pca_data)

In [ ]:
oOne.tools[2].gmm_cluster_data

In [ ]:
gmm_cluster_data=result3.get('response',{}).get('data','{}')

In [ ]:
result2.get('response',{}).get('meta_data','{}')

In [ ]:
pd.DataFrame(json.loads(gmm_cluster_data).get('records'))

In [ ]:
result4=oOne.tools[1].plot_scatter_3d(x='ILO-1', y='ILO-2', z='ILO-4', hover_data=['ENO/Title & First Name','Grade'], color='cluster',data=gmm_cluster_data)

In [ ]:
HTML(json.loads(result1.get('response',{}).get('data',{})).get('figure'))

In [ ]:
result1=oOne.get_tool_method('plot_correlogram')(00,cols=['ILO-1','ILO-2','ILO-3','ILO-4'], method="pearson", data=ILO_mapped_data)

In [ ]:
dir(oOne.tools[5])

In [ ]:
aa=oOne.tools[5].enable_google_form_email_collection(form_id='1q2Or8FBuWI-7wyZrTKuHlzB-L5CYNdZwomMaX2HZ7n4')

In [ ]:
result=oOne.tools[5].create_google_form(title='Test-2-4m-Colab',email_collection_mode='VERIFIED')

In [ ]:
result

In [ ]:
questions=[
            {
            "createItem": {
                "item": {
                "title": "Your favorite color2?",
                "questionItem": {
                    "question": {
                    "required": True,
                    "choiceQuestion": {
                        "type": "RADIO",
                        "options": [{"value": "Red"},
                                    {"value": "Blue"},
                                    {"value": "Green"}],
                        "shuffle": False
                    }
                    }
                }
                },
                "location": {"index": 0}
            }
            }
        ]
result2=oOne.tools[5].add_questions_to_google_form(form_id='1q2Or8FBuWI-7wyZrTKuHlzB-L5CYNdZwomMaX2HZ7n4',questions=questions)

In [ ]:
result2

In [ ]:
result5=oOne.tools[5].get_google_form_responses(form_id='1q2Or8FBuWI-7wyZrTKuHlzB-L5CYNdZwomMaX2HZ7n4')

In [ ]:
pd.DataFrame(json.loads(result5.get('response',{}).get('data')).get('records'))